In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input,Flatten, Dropout,GlobalMaxPooling2D,Lambda

from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

import wandb

2023-03-24 07:47:52.644739: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 07:47:52.755908: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-24 07:47:52.782288: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a suppor

In [ ]:
dir_list = sorted(os.listdir("./CUB_200_2011/images/"))
dir_list

['001.Black_footed_Albatross',
 '002.Laysan_Albatross',
 '003.Sooty_Albatross',
 '004.Groove_billed_Ani',
 '005.Crested_Auklet',
 '006.Least_Auklet',
 '007.Parakeet_Auklet',
 '008.Rhinoceros_Auklet',
 '009.Brewer_Blackbird',
 '010.Red_winged_Blackbird',
 '011.Rusty_Blackbird',
 '012.Yellow_headed_Blackbird',
 '013.Bobolink',
 '014.Indigo_Bunting',
 '015.Lazuli_Bunting',
 '016.Painted_Bunting',
 '017.Cardinal',
 '018.Spotted_Catbird',
 '019.Gray_Catbird',
 '020.Yellow_breasted_Chat',
 '021.Eastern_Towhee',
 '022.Chuck_will_Widow',
 '023.Brandt_Cormorant',
 '024.Red_faced_Cormorant',
 '025.Pelagic_Cormorant',
 '026.Bronzed_Cowbird',
 '027.Shiny_Cowbird',
 '028.Brown_Creeper',
 '029.American_Crow',
 '030.Fish_Crow',
 '031.Black_billed_Cuckoo',
 '032.Mangrove_Cuckoo',
 '033.Yellow_billed_Cuckoo',
 '034.Gray_crowned_Rosy_Finch',
 '035.Purple_Finch',
 '036.Northern_Flicker',
 '037.Acadian_Flycatcher',
 '038.Great_Crested_Flycatcher',
 '039.Least_Flycatcher',
 '040.Olive_sided_Flycatcher',
 '

In [ ]:
x_train, x_test = [],[]
y_train, y_test = [],[]

for i in dir_list:
    cnt = 0 
    for j in os.listdir("./CUB_200_2011/images/"+i):
        img = plt.imread("./CUB_200_2011/images/"+i+"/"+j)
        y = int(i.split(".")[0])
        
        if cnt<9:
            x_test.append(img)
            y_test.append(y)
            cnt += 1
        else:
            x_train.append(img)
            y_train.append(y)

In [ ]:
plt.imshow(img) 
print('class:', y)

In [ ]:
def preprocessing(image):
    resized_image = tf.image.resize(image, [112,112])
    return tf.keras.applications.vgg16.preprocess_input(resized_image)
    #return tf.keras.applications.densenet.preprocess_input(resized_image)

X_train = []

for _ in x_train:
    if len(img.shape) == 3 or len(img.shape) == 4:
        img = preprocessing(img)
        X_train.append(img)
        
X_train = np.array(X_train)
y_train = np.array(y_train)
y_train = tf.keras.utils.to_categorical(y_train)



X_test = []

for _ in x_test:
    if len(img.shape) == 3 or len(img.shape) == 4:
        img = preprocessing(img)
        X_test.append(img)
        
X_test = np.array(X_test)
y_test = np.array(y_test)       
y_test = tf.keras.utils.to_categorical (y_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Validation set 생성

In [ ]:
from sklearn.model_selection import train_test_split

X_tn, X_val, y_tn, y_val = train_test_split(X_train, y_train, test_size=0.18, random_state=42)

In [ ]:
print(X_tn.shape) 
print(y_tn.shape) 
print(X_test.shape)
print(y_test.shape)
print(X_val.shape) 
print(y_val.shape)

In [15]:
wandb.init(project="Bird_DenseNet", entity="hcim", name='DenseNet_169')

categorical_accuracy,▁▂▂▃▃▃▅▄▅▅▅▅▅▄▅▆▇▅▆▆▅▇▇▇▆▇▆▅▇▇▇▇▇▇▇██▇▇▇
loss,█▆▅▄▅▄▃▃▃▃▃▃▃▃▂▃▂▂▂▂▃▂▂▂▂▂▂▃▂▂▂▁▂▁▁▁▁▁▂▂
val_categorical_accuracy,▁▂▂▃▂▄▅▂▆▄▄▆▆▅▅▇▅▆▆█▇▇▇▆▆▆▆▅██▇█▆▆▆▆▆▆▆▇
val_loss,█▄▄▄▃▂▃▃▃▂▂▂▂▂▂▁▂▂▂▁▁▁▁▂▁▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁
categorical_accuracy,0.14493
loss,2.44687
val_categorical_accuracy,0.23582
val_loss,2.31915


# DenseNet으로 Model 생성

In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Conv2D,AveragePooling2D, GlobalAveragePooling2D, Flatten, Dense, Activation, MaxPool2D, BatchNormalization, Dropout,RandomFlip,RandomRotation

# 각 bottleneck layer 
def conv_block(x, growth_rate):
    x1 = BatchNormalization()(x)
    x1 = Activation('relu')(x1)
    x1 = Conv2D(4 * growth_rate, (1, 1), use_bias=False, padding='same')(x1)  #1x1 conv로 input feature map을 압축해 channel수를 줄임
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x1 = Conv2D(growth_rate, (3, 3), use_bias=False, padding='same')(x1) # grow_rate출력channel로 3x3conv을 이용해 new feature map 생성
    x = tf.keras.layers.Concatenate()([x, x1])  # 원래 input feature와 새 feature map 연결
    return x

# DenseNet의 Dense block
def dense_block(x, blocks, growth_rate):  # x는 입력, blocks는 Denseblock의 layer수, growth_rate는 각 dense layer의 output feature map의 개수인 dense block의 성장률을 나타내는 float값
    for i in range(blocks):
        x = conv_block(x, growth_rate)
    return x

# dense block 뒤의 transition layer_특징맵 및 입력의 공간의 차원을 줄이기 위해 사용 
def transition_block(x, reduction):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(int(tf.keras.backend.int_shape(x)[3] * reduction), (1, 1), use_bias=False, padding='same')(x)  # 수치적 안정성을 향상시키기 위해 use_bias=False로 설정
    # reduction을 channel수에 곱하여 convolution계층의 출력 channel수를 만든다.(즉, 출력channel수를 제어할 수 있음)
    x = AveragePooling2D((2, 2), strides=(2, 2))(x)
    return x

# 전체적인 모델 구성
def densenet(blocks=[6, 12, 32, 32], growth_rate=32, include_top=True, weights=None, input_shape=None, classes=1000):
    inputs = Input(shape=input_shape)
    x = tf.keras.layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(inputs) #출력 피처 맵이 입력과 동일한 공간 차원을 갖도록하는 zero-padding사용 => 정보손실방지
    x = Conv2D(64, (7, 7), strides=(2, 2), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = tf.keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x) #conv층 이후에도 출력 피처 맵이 입력과 동일한 공간 차원을 갖도록하는 zero-padding사용 => 정보손실방지
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # dense block 및 transition block 추가
    for i, num_blocks in enumerate(blocks):  # 입력한 층의 개수에 대해 접근  // (0, 6) (1, 12) (2, 32) (3, 32)
        x = dense_block(x, num_blocks, growth_rate)
        if i != len(blocks)-1: # 층의 개수에 도달하기 전까지 계속 추가 (다만 D->T->D->T->...->Dense로 끝나기에 -1을 해줘야 함)
            x = transition_block(x, 0.5)  #growth_rate를 0.5로 설정

    if include_top:
        x = GlobalAveragePooling2D()(x)
        x = Dense(classes, activation='softmax')(x)
        
    model = Model(inputs, x, name='densenet')
    return model

model = densenet(input_shape=(112,112,3), classes=201)
model.summary()

Model: "densenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 118, 118, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_168 (Conv2D)            (None, 56, 56, 64)   9408        ['zero_padding2d_2[0][0]']       
                                                                                           

In [17]:
from tensorflow.keras import callbacks
class WandbCallback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        wandb.log(logs)
        
        

from keras.preprocessing.image import ImageDataGenerator

#datagen = ImageDataGenerator(rotation_range=15, horizontal_flip=True, vertical_flip=True, brightness_range=[0.01, 0.3], channel_shift_range=30)
datagen = ImageDataGenerator(rotation_range=15, horizontal_flip=True)
datagen.fit(X_tn)

In [19]:
optimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=10e-4)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[tf.keras.metrics.CategoricalAccuracy()]) 

hist = model.fit_generator(datagen.flow(X_tn, y_tn, batch_size=128), epochs=3000, validation_data=(X_val, y_val), callbacks=[WandbCallback()])

/tmp/ipykernel_37729/3989902.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(datagen.flow(X_tn, y_tn, batch_size=128), epochs=3000, validation_data=(X_val, y_val), callbacks=[WandbCallback()])


Epoch 1/3000
64/64 [==============================] - 31s 332ms/step - loss: 4.1709 - categorical_accuracy: 0.0454 - val_loss: 72.7251 - val_categorical_accuracy: 0.0195
Epoch 2/3000
64/64 [==============================] - 20s 304ms/step - loss: 3.5712 - categorical_accuracy: 0.0600 - val_loss: 38.3550 - val_categorical_accuracy: 0.0245
Epoch 3/3000
64/64 [==============================] - 19s 294ms/step - loss: 3.3052 - categorical_accuracy: 0.0786 - val_loss: 22.7979 - val_categorical_accuracy: 0.0378
Epoch 4/3000
64/64 [==============================] - 19s 299ms/step - loss: 3.1884 - categorical_accuracy: 0.0805 - val_loss: 13.2660 - val_categorical_accuracy: 0.0378
Epoch 5/3000
64/64 [==============================] - 19s 296ms/step - loss: 3.2677 - categorical_accuracy: 0.0774 - val_loss: 22.3800 - val_categorical_accuracy: 0.0462
Epoch 6/3000
64/64 [==============================] - 19s 297ms/step - loss: 3.0759 - categorical_accuracy: 0.0780 - val_loss: 6.6781 - val_categorica

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



64/64 [==============================] - 19s 296ms/step - loss: 2.0965 - categorical_accuracy: 0.1924 - val_loss: 1.7492 - val_categorical_accuracy: 0.3142
Epoch 105/3000
64/64 [==============================] - 19s 295ms/step - loss: 2.1023 - categorical_accuracy: 0.1901 - val_loss: 2.0091 - val_categorical_accuracy: 0.2247
Epoch 106/3000
64/64 [==============================] - 19s 301ms/step - loss: 2.0774 - categorical_accuracy: 0.1944 - val_loss: 1.7933 - val_categorical_accuracy: 0.2475
Epoch 107/3000
64/64 [==============================] - 19s 298ms/step - loss: 1.9855 - categorical_accuracy: 0.2156 - val_loss: 1.8478 - val_categorical_accuracy: 0.3437
Epoch 113/3000
64/64 [==============================] - 19s 301ms/step - loss: 1.9267 - categorical_accuracy: 0.2325 - val_loss: 1.6589 - val_categorical_accuracy: 0.3309
Epoch 114/3000
64/64 [==============================] - 19s 291ms/step - loss: 1.9229 - categorical_accuracy: 0.2190 - val_loss: 1.7556 - val_categorical_accura

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

64/64 [==============================] - 19s 297ms/step - loss: 1.1658 - categorical_accuracy: 0.5510 - val_loss: 0.9428 - val_categorical_accuracy: 0.6546
Epoch 2595/3000
64/64 [==============================] - 20s 305ms/step - loss: 1.2338 - categorical_accuracy: 0.5308 - val_loss: 0.8421 - val_categorical_accuracy: 0.7069
Epoch 2596/3000
64/64 [==============================] - 19s 294ms/step - loss: 1.1827 - categorical_accuracy: 0.5366 - val_loss: 0.9207 - val_categorical_accuracy: 0.6602
Epoch 2597/3000
64/64 [==============================] - 19s 298ms/step - loss: 1.2654 - categorical_accuracy: 0.4734 - val_loss: 0.8117 - val_categorical_accuracy: 0.7197
Epoch 2604/3000
64/64 [==============================] - 19s 297ms/step - loss: 1.1156 - categorical_accuracy: 0.5258 - val_loss: 0.8653 - val_categorical_accuracy: 0.6780
Epoch 2605/3000
64/64 [==============================] - 19s 295ms/step - loss: 1.0696 - categorical_accuracy: 0.5547 - val_loss: 0.9386 - val_categorical_a

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



64/64 [==============================] - 19s 302ms/step - loss: 1.1120 - categorical_accuracy: 0.5651 - val_loss: 0.9564 - val_categorical_accuracy: 0.6346
Epoch 2803/3000
64/64 [==============================] - 19s 300ms/step - loss: 1.2190 - categorical_accuracy: 0.4900 - val_loss: 0.8999 - val_categorical_accuracy: 0.6563
Epoch 2804/3000
64/64 [==============================] - 20s 302ms/step - loss: 1.0986 - categorical_accuracy: 0.5353 - val_loss: 0.8591 - val_categorical_accuracy: 0.7303
Epoch 2810/3000
64/64 [==============================] - 20s 304ms/step - loss: 1.1557 - categorical_accuracy: 0.5541 - val_loss: 0.7920 - val_categorical_accuracy: 0.7286
Epoch 2811/3000
64/64 [==============================] - 19s 300ms/step - loss: 1.1330 - categorical_accuracy: 0.5375 - val_loss: 0.8866 - val_categorical_accuracy: 0.6719
Epoch 2812/3000
64/64 [==============================] - 19s 293ms/step - loss: 1.0355 - categorical_accuracy: 0.5650 - val_loss: 1.0093 - val_categorical_a

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



64/64 [==============================] - 19s 295ms/step - loss: 1.3936 - categorical_accuracy: 0.4860 - val_loss: 0.7494 - val_categorical_accuracy: 0.7653
Epoch 2908/3000
64/64 [==============================] - 19s 299ms/step - loss: 1.0209 - categorical_accuracy: 0.5875 - val_loss: 0.7823 - val_categorical_accuracy: 0.7464
Epoch 2909/3000
64/64 [==============================] - 19s 301ms/step - loss: 1.1313 - categorical_accuracy: 0.5637 - val_loss: 0.8134 - val_categorical_accuracy: 0.7119
Epoch 2910/3000
64/64 [==============================] - 19s 302ms/step - loss: 1.2234 - categorical_accuracy: 0.5440 - val_loss: 0.8395 - val_categorical_accuracy: 0.7024
Epoch 2916/3000
64/64 [==============================] - 20s 303ms/step - loss: 1.1819 - categorical_accuracy: 0.4918 - val_loss: 0.8686 - val_categorical_accuracy: 0.6891
Epoch 2917/3000
64/64 [==============================] - 19s 298ms/step - loss: 1.3985 - categorical_accuracy: 0.4927 - val_loss: 0.8864 - val_categorical_a

In [22]:
print(model.evaluate(X_tn, y_tn))
print(model.evaluate(X_val, y_val))

256/256 [==============================] - 8s 33ms/step - loss: 0.8655 - categorical_accuracy: 0.7042
[0.8654647469520569, 0.7041513919830322]
57/57 [==============================] - 2s 33ms/step - loss: 0.8598 - categorical_accuracy: 0.7030
[0.859761655330658, 0.7030033469200134]


In [ ]:
wandb.finish()